# IMDB Movie Data

This is a summary dataset that contains all the data of IMDB from 100,000 ratings and 3,600 tag applications applied to 9,000 movies by 600 users

https://grouplens.org/datasets/movielens/latest

## Support Vector Classification Machines

Hypothesis: Can we predict whether a movie will be "Good" or "Bad' using the following variables Genres, Popularity and Revenue?

In [1]:
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd
import numpy as np
import os

import json
# Python ≥3.5 is required
import sys
#assert sys.version_info >= (3, 5)
# Scikit-Learn ≥0.20 is required
import sklearn
#assert sklearn.__version__ >= "0.20"
# Common imports
import os # operator system
# to make this notebook's output stable across runs
np.random.seed(42)

In [2]:
 # Read the csv file into a pandas DataFrame
data = pd.read_csv("../Resources/data/train.csv")
data.head()

,id,belongs_to_collection,budget,genres,homepage,imdb_id,original_language,original_title,overview,popularity,...,release_date,runtime,spoken_languages,status,tagline,title,Keywords,cast,crew,revenue
0,1,"[{'id': 313576, 'name': 'Hot Tub Time Machine ...",14000000,"[{'id': 35, 'name': 'Comedy'}]",NaN,tt2637294,en,Hot Tub Time Machine 2,"When Lou, who has become the ""father of the In...",6.575393,...,2/20/15,93.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The Laws of Space and Time are About to be Vio...,Hot Tub Time Machine 2,"[{'id': 4379, 'name': 'time travel'}, {'id': 9...","[{'cast_id': 4, 'character': 'Lou', 'credit_id...","[{'credit_id': '59ac067c92514107af02c8c8', 'de...",12314651
1,2,"[{'id': 107674, 'name': 'The Princess Diaries ...",40000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,tt0368933,en,The Princess Diaries 2: Royal Engagement,Mia Thermopolis is now a college graduate and ...,8.248895,...,8/6/04,113.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,It can take a lifetime to find true love; she'...,The Princess Diaries 2: Royal Engagement,"[{'id': 2505, 'name': 'coronation'}, {'id': 42...","[{'cast_id': 1, 'character': 'Mia Thermopolis'...","[{'credit_id': '52fe43fe9251416c7502563d', 'de...",95149435
2,3,NaN,3300000,"[{'id': 18, 'name': 'Drama'}]",http://sonyclassics.com/whiplash/,tt2582802,en,Whiplash,"Under the direction of a ruthless instructor, ...",64.299990,...,10/10/14,105.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The road to greatness can take you to the edge.,Whiplash,"[{'id': 1416, 'name': 'jazz'}, {'id': 1523, 'n...","[{'cast_id': 5, 'character': 'Andrew Neimann',...","[{'credit_id': '54d5356ec3a3683ba0000039', 'de...",13092000
3,4,NaN,1200000,"[{'id': 53, 'name': 'Thriller'}, {'id': 18, 'n...",http://kahaanithefilm.com/,tt1821480,hi,Kahaani,Vidya Bagchi (Vidya Balan) arrives in Kolkata ...,3.174936,...,3/9/12,122.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,NaN,Kahaani,"[{'id': 10092, 'name': 'mystery'}, {'id': 1054...","[{'cast_id': 1, 'character': 'Vidya Bagchi', '...","[{'credit_id': '52fe48779251416c9108d6eb', 'de...",16000000
4,5,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",NaN,tt1380152,ko,마린보이,Marine Boy is the story of a former national s...,1.148070,...,2/5/09,118.0,"[{'iso_639_1': 'ko', 'name': '한국어/조선말'}]",Released,NaN,Marine Boy,NaN,"[{'cast_id': 3, 'character': 'Chun-soo', 'cred...","[{'credit_id': '52fe464b9251416c75073b43', 'de...",3923970


In [3]:
data = data[['id', 'budget', 'genres', 'popularity', 'cast', 'revenue']]

In [5]:
data['revenue']=data['revenue'].fillna(0)
data

,id,budget,genres,popularity,cast,revenue
0,1,14000000,"[{'id': 35, 'name': 'Comedy'}]",6.575393,"[{'cast_id': 4, 'character': 'Lou', 'credit_id...",12314651
1,2,40000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",8.248895,"[{'cast_id': 1, 'character': 'Mia Thermopolis'...",95149435
2,3,3300000,"[{'id': 18, 'name': 'Drama'}]",64.299990,"[{'cast_id': 5, 'character': 'Andrew Neimann',...",13092000
3,4,1200000,"[{'id': 53, 'name': 'Thriller'}, {'id': 18, 'n...",3.174936,"[{'cast_id': 1, 'character': 'Vidya Bagchi', '...",16000000
4,5,0,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",1.148070,"[{'cast_id': 3, 'character': 'Chun-soo', 'cred...",3923970
...,...,...,...,...,...,...
2995,2996,0,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...",9.853270,"[{'cast_id': 2, 'character': 'Rock Reilly', 'c...",1596687
2996,2997,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10402, 'n...",3.727996,"[{'cast_id': 5, 'character': 'Bobo', 'credit_i...",180590
2997,2998,65000000,"[{'id': 80, 'name': 'Crime'}, {'id': 28, 'name...",14.482345,"[{'cast_id': 10, 'character': 'Samantha Caine ...",89456761
2998,2999,42000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...",15.725542,"[{'cast_id': 8, 'character': 'Reuben Feffer', ...",171963386


In [4]:
def get_lists(row):
    try:
        return eval(row)
    except:
        return None
    
genre = data[['id','genres']]
genre['genres'] = genre['genres'].apply(get_lists)
genre = genre.explode('genres')

def get_name(row):
    try:
        return row['name']
    except:
        return None

genre['genre_name']=genre['genres'].apply(get_name)

genre = genre[["id", "genre_name"]]
genre_dummies = pd.get_dummies(genre, prefix="",prefix_sep="",columns=["genre_name"])
genre_dummies.groupby("id").sum()

<ipython-input-4-fab75685d405>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  genre['genres'] = genre['genres'].apply(get_lists)


,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,Foreign,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western
id,,,,,,,,,,,,,,,,,,,,
1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,1,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0
3,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0
5,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2996,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
2997,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0
2998,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0


In [6]:
merged = pd.merge(data, genre_dummies, on="id")
merged.head()

,id,budget,genres,popularity,cast,revenue,Action,Adventure,Animation,Comedy,...,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western
0,1,14000000,"[{'id': 35, 'name': 'Comedy'}]",6.575393,"[{'cast_id': 4, 'character': 'Lou', 'credit_id...",12314651,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,2,40000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",8.248895,"[{'cast_id': 1, 'character': 'Mia Thermopolis'...",95149435,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,2,40000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",8.248895,"[{'cast_id': 1, 'character': 'Mia Thermopolis'...",95149435,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2,40000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",8.248895,"[{'cast_id': 1, 'character': 'Mia Thermopolis'...",95149435,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2,40000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",8.248895,"[{'cast_id': 1, 'character': 'Mia Thermopolis'...",95149435,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [17]:
# add definition for 'good/bad' movie 
merged['word_rating'] = np.where(merged['popularity'] >5, "Good", "Bad")


In [25]:
# pick columns for x 

x_df = merged[['revenue', 'Action', 'Adventure', 'Animation', 
                                          'Comedy', 'Crime', 'Documentary', 'Drama', 'Family', 
                                          'Fantasy', 'Foreign', 'History', 'Horror', 'Music', 
                                          'Mystery', 'Romance', 'Science Fiction', 'TV Movie', 
                                          'Thriller', 'War', 'Western']]

x_df = x_df.reset_index(drop=True)

x_df

,revenue,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,...,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western
0,12314651,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,95149435,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,95149435,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,95149435,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,95149435,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7513,171963386,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7514,171963386,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
7515,82087155,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
7516,82087155,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
# Assign the data to X
# X = pd.get_dummies(data=merged,columns=[['revenue', 'Action', 'Adventure', 'Animation', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Family', 'Fantasy', 'Foreign', 'History', 'Horror', 'Music', 'Mystery', 'Romance', 'Science Fiction', 'TV Movie', 'Thriller', 'War', 'Western']])
# X


X = x_df[['revenue','Action', 'Adventure', 'Animation', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Family', 'Fantasy', 'Foreign', 'History', 'Horror', 'Music', 'Mystery', 'Romance', 'Science Fiction', 'TV Movie', 'Thriller', 'War', 'Western']]

In [27]:
X


,revenue,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,...,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western
0,12314651,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,95149435,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,95149435,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,95149435,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,95149435,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7513,171963386,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7514,171963386,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
7515,82087155,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
7516,82087155,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
# Assign the data to y (47 Variables which are going to predict the Revenue)
y = merged["word_rating"]

In [29]:
# Use train_test_split to create training and testing data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
# Support vector machine linear classifier
from sklearn.svm import SVC 
model = SVC(kernel='linear')
model.fit(X_train, y_train)

In [ ]:
# Model Accuracy
print('Test Acc: %.3f' % model.score(X_test, y_test))

In [ ]:
# Calculate classification report
from sklearn.metrics import classification_report
predictions = model.predict(X_test)
print(classification_report(y_test, predictions))

## Neural Network